# Clustering Algorithms - K-Means Demo and Exercise

Now that you have been introduced to clustering algorithms, we will demonstrate how it is done in Python and provide a few practice exercises.

## Introduction
The k-means algorithm searches for a pre-determined number of clusters within an unlabeled multidimensional dataset. It accomplishes this using a simple conception of what the optimal clustering looks like:
- The "cluster center" is the arithmetic mean of all the points belonging to the cluster.
- Each point is closer to its own cluster center than to other cluster centers.

Those two assumptions are the basis of the k-means model. 

We will soon dive into exactly how the algorithm reaches this solution, but for now let's take a look at a simple dataset and see the k-means result.

### Generate the dataset and plot for visual inspection
We can probably pick out the clusters visually if we plot them in the feature space. In this example, we will generate a dataset with two features and 300 samples. Once it's generated, we will plot the dataset to visually inspect the clusters.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

In [ ]:
from sklearn.datasets import make_blobs

X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

### Cluster with k-means and plot again for validation
Based on visual inspection, the appropriate number of clusters appears to be 4. We will fit the `KMeans` model from `sklearn.cluster` to our feature data and then plot the predicted clusters and their centroids.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

### Evaluation and Tuning
In order to determine the optimal number of clusters, we can try different values and measure the quality of our clusters with each iteration and select the model with the highest quality.

#### Evaluation Metrics
We discussed a few measures of cluster quality we can use to assess our clusters in a more objective way, including:

- Within clusters:
    - __Inertia__: Measures how close objects are to their cluster centroid
    - __Cohesion__: Measures how close objects are to other members in their cluster

- Between clusters
    - __Separation__: Measures how distinct clusters are from one another

- A combination of measurements:
    - __Silhouette__: Based on a combination of Cohesion and Separation
    
We will access some metrics from the model directly, and for others we will use `sklearn.metrics`.

In [ ]:
print("Inertia: " + str(kmeans.inertia_))

What you want is a model with a low intertia AND low number of clusters, but that is a tradeoff.

In [ ]:
from sklearn.metrics import silhouette_score

s_score = silhouette_score(X, y_kmeans)
print("Silhouette: " + str(s_score))

For silhoutte score, we want a value that's closer to one than negative one.

#### Tuning
We can try different values for the `n_clusters` parameter and measure the quality with each iteration to determine the optimal value and confirm our visual assessment.

It's helpful to plot the results of each iteration to see the impact on each metric.

In [ ]:
inertia = []
sil = []

# changing the number of clusters 
for k in range(2,11):
    
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(X)
    y_pred = km.predict(X)
    
    inertia.append((k, km.inertia_))
    sil.append((k, silhouette_score(X, y_pred)))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))

# Plotting Elbow Curve
x_iner = [x[0] for x in inertia]
y_iner  = [x[1] for x in inertia]
ax[0].plot(x_iner, y_iner)
ax[0].set_xlabel('Number of Clusters')
ax[0].set_ylabel('Intertia')
ax[0].set_title("Inertia Score - AKA. 'Elbow Curve'")

# Plotting Silhouetter Score
x_sil = [x[0] for x in sil]
y_sil  = [x[1] for x in sil]
ax[1].plot(x_sil, y_sil)
ax[1].set_xlabel('Number of Clusters')
ax[1].set_ylabel('Silhouetter Score')
ax[1].set_title('Silhouetter Score Curve')

#### Final assessment
The silouette score confirms our visual evaluation that the optimal number of clusters for this particular dataset appears to be 4, which makes sense since it was generated as such at the start of the demonstration.

It's important to understand that the intertia score will continue to improve as we break the data down into smaller and smaller clusters. However, there is an obvious point at Number of Clusters equal to 4 where the improvement in the intertia metric starts to level off. This is known as the _elbow_ and is the optimal point from the intertia perspective.

Finally, we can take a look at each prediction for each observation.

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_pred = kmeans.predict(X)
y_pred

### Limitations
The fundamental model assumptions of k-means (points will be closer to their own cluster center than to others) means that the algorithm will often be ineffective if the clusters have complicated geometries.

In particular, the boundaries between k-means clusters will always be linear, which means that it will fail for more complicated boundaries. Consider the following data, along with the cluster labels found by the typical k-means approach:

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(200, noise=.05, random_state=0)

In [ ]:
labels = KMeans(2, random_state=0).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

### Popular Use Case: Image Compression

One interesting application of clustering is in color compression within images. For example, imagine you have an image with millions of colors. In most images, a large number of the colors will be unused, and many of the pixels in the image will have similar or even identical colors.

#### Context
Images contain many different colours in each pixel, which can be stored as different combinations of the colours red, green and blue (AKA. RGB values). RGB values are measured as integers between 0 and 255, which means there are a total of `255 * 255 * 255` or 16 million possible colours. Ultimately, the size of the image will depend in part on the number of different colours present in the image. 

#### The goal
If we reduce the number of colours in the image, less information will be stored and thus the image will be smaller. We may lose some information in the image, but as long as we can still tell what the image is, that's okay.

#### The solution
We can use KMeans clustering to group similar colours, and then replace all the colours with the colour at their cluster's centre, thus compressing the image while also preserving it.

#### The process

__Step 1__: Import and plot the original image
- We will use a sample image from `sklearn.datasets` to demonstrate.
- We will plot the image and then vectorize it into a matrix of pixels and RGB (Red, Green, Blue) values for each pixel.

In [ ]:
from sklearn.datasets import load_sample_image

china = load_sample_image("china.jpg")
fig = plt.figure(figsize=(24,18))
ax1 = fig.add_subplot(2,2,1)
ax1.axis('off')
ax1.imshow(china, aspect='auto');

__Step 2:__ Inspect matrix of pixels (observations) and their Red, Green, and Blue values (features).

We will show the first few pixels to get an idea of what the matrix looks like.
- It has 427 rows, 640 columns, and three values for each pixel

In [ ]:
china.shape

In [ ]:
china[10]

__Step 3:__ Reshaping and scaling the data.
- reshape the data to `[n_samples x n_features]`
    - Each row will be a pixel
    - Each column will be the red, green, or blue value
- rescale the RGB values so that they lie between 0 and 1
    - Divide by 255

In [ ]:
data = china / 255.0 # use 0...1 scale
data = data.reshape(427 * 640, 3)
data.shape

In [ ]:
data

__Step 4:__ Plot the data in the feature space.

Once we have transformed and scaled the data, we can plot it. Since we have three features and we are plotting in 2D, we will need to examine two plots.

We can show all the pixels in the photo based on the extent to which they express each feature (Red, Green or Blue).

In [ ]:
def plot_pixels(data, title, colors=None, N=10000):
    if colors is None:
        colors = data
    
    # choose a random subset
    rng = np.random.RandomState(0)
    i = rng.permutation(data.shape[0])[:N]
    colors = colors[i]
    R, G, B = data[i].T
    
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    ax[0].scatter(R, G, color=colors, marker='.')
    ax[0].set(xlabel='Red', ylabel='Green', xlim=(0, 1), ylim=(0, 1))

    ax[1].scatter(R, B, color=colors, marker='.')
    ax[1].set(xlabel='Red', ylabel='Blue', xlim=(0, 1), ylim=(0, 1))

    fig.suptitle(title, size=20);

In [ ]:
plot_pixels(data, title='Input color space: 16 million possible colors')

In [ ]:
255 * 255 * 255

__Step 5:__ Reducing the number of colours down to 16 using `KMeans` and plotting the results

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=16)
kmeans.fit(data)
new_colors = kmeans.cluster_centers_[kmeans.predict(data)]

#Plotting the results
plot_pixels(data, colors=new_colors,
            title="Reduced color space: 16 colors")

__Step 6:__ Compare results before and after compression

Plotting these new colors in the image space rather than the pixel space shows us the effect of this:

In [ ]:
china_recolored = new_colors.reshape(china.shape)

fig, ax = plt.subplots(1, 2, figsize=(16, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)
ax[0].imshow(china)
ax[0].set_title('Original Image', size=16)
ax[1].imshow(china_recolored)
ax[1].set_title('16-color Image', size=16);

---------------------------------------
# Group Exercise
We will practice appling KMeans clustering to another use-case and dataset.

Let's use the `2016-2019-voter-data.csv` and try to cluster the different municipalities based on any features available in or derived from the dataset. We want to identify distinct groups of municipalities so we can design a better program for voter education in the country.

#### The solution
We can use KMeans clustering to group similar municipalities. The optimal number of groups will be determined by trying different values and measuring the quality of the groups based on the silhouette score.

#### The process
__Step 1__: Import the dataset _(done for you)_

In [ ]:
import pandas as pd

df = pd.read_csv('data/2016-2019-voter-data.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
df.columns

__Step 2__: Select the columns you want to use for clustering and do feature scaling using `Normalizer` from `sklearn.preprocessing`.

In [ ]:
from sklearn.preprocessing import Normalizer

# Select features for clustering
feature_cols = []
X = df[feature_cols]

# Feature scaling
X=Normalizer().fit_transform(X.values)

__Step 3:__ Train initial KMeans model with `n_clusters` = 5, and print out the `inertia` and `silhouette_score`

In [ ]:
from sklearn.cluster import KMeans

# Insert code here

In [ ]:
print("Inertia: " + str(kmeans.inertia_))

In [ ]:
from sklearn.metrics import silhouette_score

s_score = silhouette_score(X, y_kmeans)
print("Silhouette: " + str(s_score))

__Step 4:__ Tune the model by trying different values for `n_clusters` between 3-20 and plot the results to determine the best parameter.

In [ ]:
# Try fitting w/ different number of clusters 

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))

# Plotting Elbow Curve

# Plotting Silhouetter Score

__Step 5:__ Apply the optimal number of clusters and append cluster labels to your original df.

In [ ]:
# Complete code here
kmeans = 
kmeans.fit(X)
labels = kmeans.predict(X)

In [ ]:
df['Cluster Labels'] = labels

__Step 6:__ Inspect which municipalities were assigned to each group when we applied the optimal number of clusters.

In [ ]:
# Insert code here